# 快速入门：MindPandas处理CSV数据

本章节以一个CSV原生数据处理流程为例子，实现数据分析的常见流程。

## 下载并读取数据

数据预处理对于模型训练非常重要，好的数据可以有效提高训练精度和效率。
本章节的示例数据集在[url]下载，解压到指定位置并进行数据处理。
下面是将CSV文件读取到DataFrame的数据格式里：

In [1]:
import hashlib

import numpy as np
import pandas as pd
import mindpandas as pd

pd.set_adaptive_concurrency(False)
pd.set_concurrency_mode("multithread")
pd.set_partition_shape((16, 3))

file_name = "data/data.csv"
mdf = pd.read_csv(file_name)

设置了后端为multithread模式，切片的维度是16*3。如需使用yr模式，将pd.set_concurrency_mode("multithread")改为pd.set_concurrency_mode("yr")。CSV原始数据输出如下：

In [2]:
print(mdf)

          col0      col1      col2      col3      col4                  col5  \
0     19265380  96764505  85593781  66614410  87156290  mlY5VQiTj8tKG9XNUPqr   
1     85493911  87828698  98232131  41549241  48135212  i0t2AEukzZVljs53ro6c   
2     28783123  64871036  95277574  42539942  30658195  f529MPhI6FCjGdJSsLYN   
3     79702669  12454864  71934373   4087017  32475645  Yx1c8kPSol5grQ06y7su   
4     56578946  62761265  43209571  96948416  80057523  ptjJvfT4bq8DYrN3QdmM   
...        ...       ...       ...       ...       ...                   ...   
9995  99836239  33382142  47424555  84061856  48850740  vmrMxVwPtg1BTYhDSlWs   
9996  72990782  90103899  69145205  33555186  54606213  eHvhtMIg6uEQl32pj5o9   
9997  82847429   9802771  40045859  82857068  41498974  wthypCb3WB06XAJYekR8   
9998  11936407  90919619  47759544  40296987  90955464  S216sTinmXeAwkvUORo5   
9999  72234883  43691775  29356767   5176795  52016923  jPfMvLmTyicb8hOGFEek   

                      col6             

CSV是一个包含10000行，48列的二维数据，index缺省情况下为自然数，columns设置为col0～col47，包含字符串，数字等数据。

## 对数据指定列进行处理

首先我们对DataFrame的"col40"列应用lambda函数，以冒号切分为列表，然后将DataFrame中的'x'值替换成pd.NaT缺省值：

In [3]:
mdf["col40"] = mdf["col40"].apply(lambda x: x.split(':') if isinstance(x, str) else [])
mdf = mdf.replace("x", pd.NaT)
print(mdf)

          col0      col1      col2      col3      col4                  col5  \
0     19265380  96764505  85593781  66614410  87156290  mlY5VQiTj8tKG9XNUPqr   
1     85493911  87828698  98232131  41549241  48135212  i0t2AEukzZVljs53ro6c   
2     28783123  64871036  95277574  42539942  30658195  f529MPhI6FCjGdJSsLYN   
3     79702669  12454864  71934373   4087017  32475645  Yx1c8kPSol5grQ06y7su   
4     56578946  62761265  43209571  96948416  80057523  ptjJvfT4bq8DYrN3QdmM   
...        ...       ...       ...       ...       ...                   ...   
9995  99836239  33382142  47424555  84061856  48850740  vmrMxVwPtg1BTYhDSlWs   
9996  72990782  90103899  69145205  33555186  54606213  eHvhtMIg6uEQl32pj5o9   
9997  82847429   9802771  40045859  82857068  41498974  wthypCb3WB06XAJYekR8   
9998  11936407  90919619  47759544  40296987  90955464  S216sTinmXeAwkvUORo5   
9999  72234883  43691775  29356767   5176795  52016923  jPfMvLmTyicb8hOGFEek   

                      col6             

我们把DataFrame的"col10"列设置为DataFrame的"label"列，并获取前100条数据，对"col3"列的数据按值进行排序后，以(16 * 3)维度分片。数据处理后如下：

In [4]:
mdf["label"] = mdf["col10"]
mdf = mdf[:-100]
mdf = mdf.sort_values("col3")
mdf.repartition((16, 3))
print(mdf)

          col0      col1      col2      col3      col4                  col5  \
5502   7992476   2107068  49620536     32310  53947723  ixQOsR4ewZqIv01ugtmH   
4341  40507421  60834497  23604476     60220  94054643  AfUV3n7XzbpQc0qyJGiY   
3997  16022005   4190446  20179593     69983  33138839  kSzfPT91K4axVHDZnbuW   
6804  66159904   5608893  13695121     72115  72179172  568oPBi1hcbfDztX3Njp   
6024  81622610  53962418  92396725     80691  90479862  xrHTlSvZgDdh36G0Pwn9   
...        ...       ...       ...       ...       ...                   ...   
7619  45303883  66283755  63917709  99960674  70961680  bFL9HpRrSz3JfKnWsBTx   
8398  87391816  37050934  24317245  99971275  75019818  basxlKqoG1V0p4DNSmY8   
7047  50248039  66470109  76502563  99979751  12283051  0NhcCIAvZUk321mOTXa9   
6919  21270905  44795699  63237498  99987881  52881984  1SFYdCopItgPrZbUv8ka   
723   44399386  91338098  87335736  99988682  44098762  hHQzYkinmDGXPuMFBWVI   

                      col6             

## 调用API进行数据转换，新增列等处理

对生成的DataFrame去掉index索引，生成新的一列，将数据前70%设为训练集，后30%设为非训练集。然后删除掉密集的特征列，将缺省值填充为-1。源码及运行结果如下:

In [5]:
mdf = mdf.reset_index(drop=True)
m_train_len = int(len(mdf) * 0.7)
mdf["is_training"] = [1] * m_train_len + [0] * (len(mdf) - m_train_len)

dense_feat_names = ["col6", "col8", "col9", "col10", "col13", "col17", "col18", "col19", "col22", "col23", "col37"]
sparse_feat_names = ["col1", "col2", "col3", "col4", "col5", "col7", "col11", "col15", "col20", "col30", "col35"]
mdf = mdf.drop(columns=dense_feat_names)
mdf = mdf.fillna("-1")
print(mdf)

          col0      col1      col2      col3      col4                  col5  \
0      7992476   2107068  49620536     32310  53947723  ixQOsR4ewZqIv01ugtmH   
1     40507421  60834497  23604476     60220  94054643  AfUV3n7XzbpQc0qyJGiY   
2     16022005   4190446  20179593     69983  33138839  kSzfPT91K4axVHDZnbuW   
3     66159904   5608893  13695121     72115  72179172  568oPBi1hcbfDztX3Njp   
4     81622610  53962418  92396725     80691  90479862  xrHTlSvZgDdh36G0Pwn9   
...        ...       ...       ...       ...       ...                   ...   
9895  45303883  66283755  63917709  99960674  70961680  bFL9HpRrSz3JfKnWsBTx   
9896  87391816  37050934  24317245  99971275  75019818  basxlKqoG1V0p4DNSmY8   
9897  50248039  66470109  76502563  99979751  12283051  0NhcCIAvZUk321mOTXa9   
9898  21270905  44795699  63237498  99987881  52881984  1SFYdCopItgPrZbUv8ka   
9899  44399386  91338098  87335736  99988682  44098762  hHQzYkinmDGXPuMFBWVI   

                      col7             

对生成的DataFrame指定列应用自定义函数，然后对"col30"列去重后，删掉index，在第20列插入"inserted"列，数据为随机生成的数值。源码及运行结果如下:

In [7]:
def hash_item(val, item_size=10000000, offset=0):
    if isinstance(val, str):
        return abs(int(hashlib.sha256(val.encode('utf-8')).hexdigest(), 16)) % item_size
    return abs(hash(val)) % item_size + offset

mdf[sparse_feat_names] = mdf[sparse_feat_names].applymap(hash_item)
mdf = mdf.drop_duplicates(subset=['col30'])
mdf = mdf.reset_index(drop=True)

np.random.seed(100)
data = np.random.rand(len(mdf))
mdf.insert(20, "inserted", pd.Series(data))
print(mdf)

          col0     col1     col2     col3     col4     col5     col7    col11  \
0      7992476  2107068  9620536    32310  3947723  8896189  2601001  8014425   
1     40507421   834497  3604476    60220  4054643  8768693  7427127  5492782   
2     16022005  4190446   179593    69983  3138839  6405246  1908999  3002804   
3     66159904  5608893  3695121    72115  2179172  6759079  8160556  1674047   
4     81622610  3962418  2396725    80691   479862    61846  4902980  8893087   
...        ...      ...      ...      ...      ...      ...      ...      ...   
9462  45303883  6283755  3917709  9960674   961680  7929077  6763613  4059688   
9463  87391816  7050934  4317245  9971275  5019818  8694335  7855541  5518791   
9464  50248039  6470109  6502563  9979751  2283051  4027226  5393130  4154283   
9465  21270905  4795699  3237498  9987881  2881984  2662542  4335469  4998567   
9466  44399386  1338098  7335736  9988682  4098762  5096226  7962388  1260445   

                     col12 

/root/anaconda3/envs/sh/lib/python3.7/site-packages/mindpandas/compiler/pandas_factory.py:953: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  return pandas.concat(data, axis)


warning为Pandas接口升级报的提示，至此，我们把原生数据通过一系列去重，应用转换函数，填充缺省值，新增数据列和分割数据集等操作，将数据处理成了可以识别的规范数据集。